## Libraries

In [16]:
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI

## Get api and connection details

### Neo4j

In [4]:
# Open the file and read the lines
with open('neo4j_connection_details.txt', 'r') as file:
    lines = file.readlines()

# Initialize variables
neo4j_uri = ""
neo4j_username = ""
neo4j_password = ""

# Process each line to extract the values
for line in lines:
    # Remove any leading/trailing whitespace
    line = line.strip()
    if line.startswith("NEO4J_URI"):
        # Extract the URI
        neo4j_uri = line.split('=')[1].strip().strip('"')
    elif line.startswith("NEO4J_USERNAME"):
        # Extract the username
        neo4j_username = line.split('=')[1].strip().strip('"')
    elif line.startswith("NEO4J_PASSWORD"):
        # Extract the password
        neo4j_password = line.split('=')[1].strip().strip('"')

# Display the extracted values
# print(f"NEO4J_URI: {neo4j_uri}")
# print(f"NEO4J_USERNAME: {neo4j_username}")
# print(f"NEO4J_PASSWORD: {neo4j_password}")


In [8]:
graph = Neo4jGraph(
    url=neo4j_uri,
    username=neo4j_username,
    password=neo4j_password,
)

In [15]:
print(graph.schema)

Node properties:
Entity {name: STRING, nameEmbedding: LIST}
Chunk {id: STRING, text: STRING, embedding: LIST}
Relationship properties:

The relationships:
(:Entity)-[:does not regulate]->(:Entity)
(:Entity)-[:defines]->(:Entity)
(:Entity)-[:regulates]->(:Entity)
(:Entity)-[:used in]->(:Entity)
(:Entity)-[:imposes obligations on]->(:Entity)
(:Entity)-[:obligates transparency to]->(:Entity)
(:Entity)-[:interacts with]->(:Entity)
(:Entity)-[:includes]->(:Entity)
(:Entity)-[:is part of]->(:Entity)
(:Entity)-[:provide]->(:Entity)
(:Entity)-[:produce]->(:Entity)
(:Entity)-[:monitors]->(:Entity)
(:Entity)-[:evaluates]->(:Entity)
(:Entity)-[:is basis for]->(:Entity)
(:Entity)-[:can be integrated into]->(:Entity)
(:Entity)-[:applies to]->(:Entity)
(:Entity)-[:helps ensure]->(:Entity)
(:Entity)-[:used by]->(:Entity)
(:Entity)-[:infer]->(:Entity)
(:Entity)-[:manages]->(:Entity)
(:Entity)-[:prioritizes]->(:Entity)
(:Entity)-[:use]->(:Entity)
(:Entity)-[:assesses risk of]->(:Entity)
(:Entity)-[:con

In [13]:
get_all_nodes = """
MATCH (n) RETURN n.name;
"""

In [14]:
graph.query(get_all_nodes)

[{'n.name': 'AI Act'},
 {'n.name': 'high-risk AI systems'},
 {'n.name': 'limited risk AI systems'},
 {'n.name': 'minimal risk AI systems'},
 {'n.name': 'generative AI'},
 {'n.name': 'third country providers'},
 {'n.name': 'AI Office'},
 {'n.name': 'GPAI model'},
 {'n.name': 'GPAI system'},
 {'n.name': 'Copyright Directive'},
 {'n.name': 'technical documentation'},
 {'n.name': 'adversarial testing'},
 {'n.name': 'real-time remote biometric identification'},
 {'n.name': 'biometric categorisation systems'},
 {'n.name': 'emotion recognition systems'},
 {'n.name': 'critical digital infrastructure'},
 {'n.name': 'employment'},
 {'n.name': 'self-employment'},
 {'n.name': 'public authorities'},
 {'n.name': 'creditworthiness'},
 {'n.name': 'emergency calls'},
 {'n.name': 'health and life insurance'},
 {'n.name': 'law enforcement'},
 {'n.name': 'migration'},
 {'n.name': 'border control management'},
 {'n.name': 'democratic processes'},
 {'n.name': 'political campaigns'},
 {'n.name': 'scientific 

### OpenAI

In [5]:
# Open the file and read the first line
with open('openai_api_key.txt', 'r') as file:
    # Read the first line and strip any leading/trailing whitespace
    openai_api_key = file.readline().strip()

# Save the value to the variable OPENAI_API_KEY
OPENAI_API_KEY = openai_api_key

# Display the extracted API key (optional, for verification)
# print(f"OPENAI_API_KEY: {OPENAI_API_KEY}")


In [48]:
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model="gpt-4o",
    temperature=0,
)

chain = GraphCypherQAChain.from_llm(
    graph=graph,
    llm=llm,
    verbose=True,
    validate_cypher=True,
)
# query = "What does law enforcement conduct? Please return the name properties but not the embeddings."  # GOOD
# query = "What do public authorities provide?"  # GOOD
# query = "What do emergency calls prioritize?"  # GOOD
query = "What do very urgent calls prioritize?"  # FAIL
# query = "What prioritizes medical aid?"  # GOOD
# query = "What is the Act about?"  # FAIL
response = chain.invoke({"query": query})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (e:Entity {name: "very urgent calls"})-[:prioritizes]->(target:Entity)
RETURN target.name

Full Context:
[]

> Finished chain.


{'query': 'What do very urgent calls prioritize?',
 'result': "I don't know the answer."}

In [37]:
?ChatOpenAI